In [12]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

engine = create_engine('sqlite:///db/projektai.db')
Base = declarative_base()
session = sessionmaker(engine)()

In [23]:
from datetime import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime

## Projektai - single table DB

In [24]:
class Projektas(Base):
    __tablename__ = "Porojektas"
    id = Column(Integer, primary_key=True)
    pavadinimas = Column("pavadinimas", String(50))
    kaina = Column("kaina", Float())
    sukurta = Column("sukurta", DateTime, default=datetime.utcnow)

    def __init__(self, pavadinimas, kaina):
        self.pavadinimas = pavadinimas
        self.kaina = kaina

    def __repr__(self):
        return f"({self.id}, {self.pavadinimas}, {self.kaina}, {self.sukurta})"


Base.metadata.create_all(engine)

/tmp/ipykernel_6141/1814235336.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Projektas, and will be replaced in the string-lookup table.
  class Projektas(Base):


InvalidRequestError: Table 'Porojektas' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

### INSERT

In [14]:
gka = Projektas("Gero Kodo Akademija", 0)
session.add(gka)
session.commit()

In [15]:
sandeliukas = Projektas("Sandeliukas 3.0", 17300)
session.add(sandeliukas)
session.commit()

In [16]:
nepaejes = Projektas("Kreiva Programulka", 700)
session.add(nepaejes)
session.commit()

In [17]:
print(gka)
print(sandeliukas)

(7, Gero Kodo Akademija, 0.0, 2023-09-18 06:49:11.453400)
(8, Sandeliukas 3.0, 17300.0, 2023-09-18 06:49:11.489112)


### SELECT'ai

In [18]:
projektas = session.get(Projektas, 2)
print(projektas)

(2, Sandeliukas 3.0, 17300.0, 2023-09-15 06:56:05.609364)


In [19]:
projektai = session.query(Projektas).all()
print(projektai)

[(1, Gero Kodo Akademija, 50000.0, 2023-09-15 06:48:41.028032), (2, Sandeliukas 3.0, 17300.0, 2023-09-15 06:56:05.609364), (4, Gero Kodo Akademija, 50000.0, 2023-09-18 06:49:08.319950), (5, Sandeliukas 3.0, 17300.0, 2023-09-18 06:49:08.346322), (6, Kreiva Programulka, 700.0, 2023-09-18 06:49:08.373705), (7, Gero Kodo Akademija, 0.0, 2023-09-18 06:49:11.453400), (8, Sandeliukas 3.0, 17300.0, 2023-09-18 06:49:11.489112), (9, Kreiva Programulka, 700.0, 2023-09-18 06:49:11.517481)]


In [20]:
nemokami = session.query(Projektas).filter(Projektas.kaina == 0).all()
print(nemokami)

[(7, Gero Kodo Akademija, 0.0, 2023-09-18 06:49:11.453400)]


### UPDATE

In [21]:
gka.kaina = 50000
session.add(gka)
session.commit()

### DELETE

In [22]:
kreiva = session.get(Projektas, 3)
session.delete(kreiva)
session.commit()

UnmappedInstanceError: Class 'builtins.NoneType' is not mapped

## Relationships (tevai/vaikai)

In [26]:
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey

In [36]:
m21_engine = create_engine('sqlite:///db/m21_tevai_vaikai.db')
m21_session = sessionmaker(m21_engine)()
BaseM21 = declarative_base()


class Vaikas(BaseM21):
    __tablename__ = "vaikas"
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    mokykla = Column("mokykla", String(50))
    tevai = relationship("Tevas", back_populates='vaikas')

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde}, {self.mokykla})"


class Tevas(BaseM21):
    __tablename__ = "tevas"
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    vaikas_id = Column(Integer, ForeignKey('vaikas.id'))
    vaikas = relationship("Vaikas", back_populates='tevai')

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde})"


BaseM21.metadata.create_all(m21_engine)

In [39]:
emilija = Vaikas(vardas="Emilija", pavarde="J", mokykla="Pasaulis")
mama = Tevas(vardas="Nico", pavarde="P", vaikas=emilija)
tevas = Tevas(vardas="Kestas", pavarde="J", vaikas=emilija)
m21_session.add_all([emilija, mama, tevas])
m21_session.commit()

In [40]:
kestas = m21_session.get(Tevas, 2)
print(kestas)

(Kestas, J)


In [42]:
tevai = m21_session.query(Tevas).filter(Tevas.vaikas == emilija).all()
print(tevai)

[(Nico, P), (Kestas, J)]


In [54]:
tevai = m21_session.query(Tevas).filter_by(vaikas=emilija).all()
print(tevai)

[(Nico, P), (Kestas, J), (Geras, G)]


In [45]:
naujas = Vaikas(vardas="Naujas", pavarde="G")
trecias = Tevas(vardas="Geras", pavarde="G", vaikas=naujas)
m21_session.add_all([naujas, trecias])
m21_session.commit()

In [48]:
visi_tevai = m21_session.query(Tevas).all()
visi_vaikai = m21_session.query(Vaikas).all()
print(visi_tevai, visi_vaikai)

[(Nico, P), (Kestas, J), (Geras, G)] [(Emilija, J, Pasaulis), (Naujas, G, None)]


In [53]:
visi_tevai[2].vaikas = emilija
m21_session.add(visi_tevai[2])
m21_session.commit()

In [55]:
ketvirtas = Tevas(vardas="Super", pavarde="Mama")
naujas.tevai.append(ketvirtas)
m21_session.add_all([ketvirtas, naujas])
m21_session.commit()

In [59]:
atnaujintas = m21_session.query(Vaikas).filter_by(vardas="Naujas").all()
print(atnaujintas)
print(atnaujintas[0].tevai)

[(Naujas, G, None)]
[(Super, Mama)]


In [61]:
supermama = m21_session.query(Tevas).filter_by(pavarde="Mama").one()
print(supermama.vaikas)

(Naujas, G, None)


## Many 2 Many tevai